In [16]:
import pandas as pd
import string
import numpy as np
import os
from pathlib import Path

data_file = Path.cwd().parent / "data" / 'FES 2023 Data Workbook V001.xlsx'

scenario_mapper = {
    "CT": "Consumer Transformation", 
    "ST": "System Transformation", 
    "LW": "Leading the Way", 
    "FS": "Falling Short",
    }

def get_distributed_generation(scenario, year):
    """Get reference and future value for commercial elecitricity demand"""

    col = string.ascii_uppercase.index("N")
    df = (
        pd.read_excel(data_file,
            sheet_name="ES.22",
            header=6,
            index_col=0,
            nrows=4,
            usecols=[col+i for i in range(30)],
            )
    )

    df.columns = [pd.Timestamp(dt).year for dt in df.columns]
    return df.loc[scenario_mapper[scenario], year] * 1e3

get_distributed_generation("LW", 2022)

                             2022       2023       2024       2025       2026  \
Consumer Transformation  31.91774  32.775736  34.603368  37.135042  39.574521   
System Transformation    31.91774  32.400937  33.595083  34.928947  36.262167   
Leading the Way          31.91774  33.805428  36.228724  39.391208  42.341785   
Falling Short            31.91774  32.877006  33.684577  34.370843  35.083741   

                              2027       2028       2029       2030  \
Consumer Transformation  42.046268  44.390456  47.015257  49.937846   
System Transformation    37.725927  39.020917  40.591371  42.655732   
Leading the Way          45.702341  48.977478  52.504994  55.011681   
Falling Short            35.769391  36.571907  37.496379  38.650723   

                              2031  ...       2041        2042        2043  \
Consumer Transformation  53.072279  ...  80.659711   83.928235   87.477583   
System Transformation    44.955182  ...  61.253916   63.022853   64.928122   
Lead

31917.739999999994

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import string

scenario_mapper = {
    "CT": "Consumer Transformation", 
    "ST": "System Transformation", 
    "LW": "Leading the Way", 
    "FS": "Falling Short",
    }


path = Path.cwd().parent / "data" / 'FES 2023 Data Workbook V001.xlsx'

def get_heating_shares(year, scenario):
    """Get reference and future value for commercial elecitricity demand"""

    col = string.ascii_uppercase.index("M")

    df = pd.read_excel(
        path,
        sheet_name="EC.E",
        header=7,
        index_col=col,
        usecols=list(range(col, col+15)),
        ).iloc[1:4]
    df.index = df.iloc[:,0]

    df_2021 = df.iloc[:,1]    

    df_2035 = (
        df.iloc[:,3:7]
        [scenario]
    )
    df_2050 = (
        df.iloc[:,8:12]
        [scenario+'.1']
    )

    get_val = lambda tech: np.interp(
        year,
        [2021, 2035, 2050],
        [df_2021.loc[tech], df_2035.loc[tech], df_2050.loc[tech]]
    )

    print(df_2035)
    return pd.Series({tech: get_val(tech) for tech in df_2035.index})


get_heating_shares(2035, "FS")

                     Unnamed: 12 Unnamed: 13  Unnamed: 14         CT  \
Unnamed: 12                                                            
Direct electric  Direct electric     2358311          NaN  2060042.0   
GSHP                        GSHP       61519          NaN  3488393.0   
ASHP                        ASHP      192862          NaN  7053537.0   

                        ST         LW         FS  Unnamed: 19        CT.1  \
Unnamed: 12                                                                 
Direct electric  1862382.0  1988387.0  1908289.0          NaN   1737630.0   
GSHP              643392.0  3500018.0   613089.0          NaN   8835201.0   
ASHP             1331053.0  8420552.0  2578076.0          NaN  14458394.0   

                      ST.1        LW.1       FS.1  Unnamed: 25  Unnamed: 26  
Unnamed: 12                                                                  
Direct electric   757939.0   1744680.0  1754150.0          NaN          NaN  
GSHP             10

Direct electric    1908289.0
GSHP                613089.0
ASHP               2578076.0
dtype: float64